# Back Order Stock Automated Ordering system.
### With any stock control system there will be times when cutomser demand outweighs the quantity of items in storage.
### The customer has to wait for their item to arrive in the Warehouse/Distribution Centre.
### From the perspective of Stock Control this is seen as a negative NET QTY.
### The purpose of this code is to take an Excel Document and identify the titles and respective quantities which require ordering.
### Also, only titles which have a Purchase Status of IP (In Print) may be ordered. This is based upon book sales.
### When complete, the code will export an Excel sheet which holds just the id and qty which require orders to be placed. This file may be copied in to SAP ordering systems.

In [10]:
import pandas as pd
import numpy as np


### Stock data is imported to python from the Excel Spreadsheet (bord_edit.xlsx)
##### From which the first 5 lines are read.
##### Stock items are coded by 'Id'
##### Purchase status indicates if book is "In Print" as IP, this will be the focus of the orders.
##### Open PO Qty is the quantity of stock in transit.
##### Total Stock is how much stock is on the shelf
##### Back Order Qty: Orders requested when stocked out.
##### Net stock = Total Stock - Back Order Qty
##### UK Prod Hier, Pub Date, Matl Grp, MPG are other fields to be looked at later.
##### The columns with date headers indicates sales per month

In [11]:
# Imports Report data
df1 = pd.read_excel("BackOrderList.xlsx")
#df1 = pd.read_csv('bord_edit.csv')
df1.head()

,Id,Purch Status,Open PO Qty,Total Stock,Back Order Qty,Net Stock,UK Prod hier,Pub Date,Matl Grp,MPG,...,mon_39,mon_40,mon_41,mon_42,mon_43,mon_44,mon_45,mon_46,mon_47,mon_48
0,1,IP,50,0,11,-11,ACAT,2017-02-01,I10,3,...,7,6,9,1,4,4,5,5,5,4
1,2,MD,0,0,1,-1,ACLAW,2016-11-15,I10,6,...,0,56,41,24,8,61,72,9,33,22
2,3,IP,1050,0,113,-113,ACDIC,2005-07-28,I10,12,...,0,9,2,11,2,15,1,0,0,8
3,4,IP,100,0,14,-14,ACAT,2008-08-01,I10,11,...,0,2,8,0,2,1,1,0,0,1
4,5,IP,0,0,1,-1,ACAT,2020-05-05,I10,6,...,0,7,7,0,12,1,3,8,0,0


### Select the books categorised as "IP" (In print)

In [12]:
df1=df1[df1['Purch Status']=='IP' ]
df1.head()

,Id,Purch Status,Open PO Qty,Total Stock,Back Order Qty,Net Stock,UK Prod hier,Pub Date,Matl Grp,MPG,...,mon_39,mon_40,mon_41,mon_42,mon_43,mon_44,mon_45,mon_46,mon_47,mon_48
0,1,IP,50,0,11,-11,ACAT,2017-02-01,I10,3,...,7,6,9,1,4,4,5,5,5,4
2,3,IP,1050,0,113,-113,ACDIC,2005-07-28,I10,12,...,0,9,2,11,2,15,1,0,0,8
3,4,IP,100,0,14,-14,ACAT,2008-08-01,I10,11,...,0,2,8,0,2,1,1,0,0,1
4,5,IP,0,0,1,-1,ACAT,2020-05-05,I10,6,...,0,7,7,0,12,1,3,8,0,0
5,6,IP,0,0,1,-1,ACAT,2020-09-09,I10,6,...,0,0,0,0,0,15,0,5,0,0


#### The 'Net Stock' column does not include stock in transit, which may mean stock is dispatched when an order is currently in place.
#### To eliminate the risk of double ordering an additional column labelled 'asanet' indicates the net qty inclusive of stock in transit. 
#### 'usanet' = 'TotalStock' + 'Open PO Qty' - 'Back Order  Qty'

In [13]:
# create an additional column named 'usanet' composed of: 'USA_TotalStock' + 'Open PO Qty' - 'BO Qty'

df1['usanet']=df1['Total Stock']+df1['Open PO Qty']-df1['Back Order Qty']
df1.head()

,Id,Purch Status,Open PO Qty,Total Stock,Back Order Qty,Net Stock,UK Prod hier,Pub Date,Matl Grp,MPG,...,mon_40,mon_41,mon_42,mon_43,mon_44,mon_45,mon_46,mon_47,mon_48,usanet
0,1,IP,50,0,11,-11,ACAT,2017-02-01,I10,3,...,6,9,1,4,4,5,5,5,4,39
2,3,IP,1050,0,113,-113,ACDIC,2005-07-28,I10,12,...,9,2,11,2,15,1,0,0,8,937
3,4,IP,100,0,14,-14,ACAT,2008-08-01,I10,11,...,2,8,0,2,1,1,0,0,1,86
4,5,IP,0,0,1,-1,ACAT,2020-05-05,I10,6,...,7,7,0,12,1,3,8,0,0,-1
5,6,IP,0,0,1,-1,ACAT,2020-09-09,I10,6,...,0,0,0,0,15,0,5,0,0,-1


### It is only required that we send stock when usanet is below a certain level, to start with we set the level as less than zero.

In [14]:
# Selects the titles with negative usanet
df2=df1[df1['usanet']<=0]
df2=df2[['Id','usanet']]
df2


,Id,usanet
4,5,-1
5,6,-1
6,7,-2
9,10,-7
24,25,-3
30,31,-5
36,37,-1
44,45,-1
48,49,-1
58,59,-1


### Assuming seasonality
#### Assuming seaonality, the average sales from the previous years are calculated as a 3month forecast.
#### Currently this is inaccurrate, but operates as a basis for a 3 month forecast. Expect further statistiacl forecast methods to be implemented here.
#### The final column is created to hold the order qty based on this estimation

In [15]:
df2['3 mon sum 2014']= (df1['mon_1']+df1['mon_2']+df1['mon_3']) 
df2['3 mon sum 2015']=(df1['mon_13']+df1['mon_14']+df1['mon_15'])
#'01 June 2014', '02 July 2014', '02 August 2014'
#quart_sales=df2[]
#df1['usanet']=df1['USA_Total_Stock']+df1['Open PO Qty']-df1['BO Qty']

df2['3m forecast']=np.round((df2['3 mon sum 2014']+df2['3 mon sum 2015'])/2,0)
df2['3m Order QTY']=df2['usanet']*(-1)+df2['3m forecast']
df2.head()


,Id,usanet,3 mon sum 2014,3 mon sum 2015,3m forecast,3m Order QTY
4,5,-1,0,14,7.0,8.0
5,6,-1,0,0,0.0,1.0
6,7,-2,0,0,0.0,2.0
9,10,-7,0,3,2.0,9.0
24,25,-3,1,17,9.0,12.0


## Produce Order sheet
#### The basic requirements for Order Sheet data are complied in  the last table.

In [16]:
Order_Sheet = df2[['Id','3m Order QTY']]
Order_Sheet


,Id,3m Order QTY
4,5,8.0
5,6,1.0
6,7,2.0
9,10,9.0
24,25,12.0
30,31,6.0
36,37,15.0
44,45,37.0
48,49,67.0
58,59,3.0


## Extract Order sheet into an Excel File named "NewOrders.xlxs"
### Which will be saved in original desktop folder.

In [17]:
Order_Sheet.to_excel(r'C:\Users\User\Desktop\OUP\Back Orders\NewOrders.xlsx', index = False)